## Installing the librarires

In [3]:
# !pip install langchain
# !pip install langchain_community
# !pip install pinecone
# !pip install langchain-google-genai
# !pip install pypdf

## Importing the dependencies

In [1]:
import langchain
import time
import os

from langchain_community.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.schema.output_parser import StrOutputParser
from langchain.prompts import PromptTemplate
from langchain_community.llms import Ollama
from langchain_community.embeddings import OllamaEmbeddings
from langchain_google_genai import ChatGoogleGenerativeAI

from pinecone import ServerlessSpec
from pinecone.grpc import PineconeGRPC as Pinecone
print("Everything improted succesfully")

Everything improted succesfully


d:\Projects\Chai-with-Youtube\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Setting up the LLM model

In [2]:
embedding_model = OllamaEmbeddings(model="jina/jina-embeddings-v2-base-de")
llm = ChatGoogleGenerativeAI(model="gemini-pro", google_api_key=os.environ['GEMINI_API'])

In [3]:
llm.invoke("What do you mean by Retrieval Augemented Generation in AI?")

AIMessage(content='**Retrieval-Augmented Generation (RAG)** is an AI technique that combines retrieval-based methods with generative models to enhance text generation tasks.\n\n**How it works:**\n\n1. **Retrieval:**\n   - RAG first retrieves relevant documents or passages from a large corpus using a retrieval model. This helps to provide the model with contextual information for the generation task.\n\n2. **Augmentation:**\n   - The retrieved documents are then used to augment the input to the generative model. This provides additional context and knowledge, allowing the model to generate more informative and coherent text.\n\n3. **Generation:**\n   - The generative model, typically a large language model (LLM), is used to generate new text based on the augmented input. The LLM learns from the retrieved documents and combines them with its own knowledge to produce high-quality text.\n\n**Advantages of RAG:**\n\n* **Improved Factual Accuracy:** Retrieval-based methods help ensure that t

## Loading the data into Vector DB

In [5]:
def chunk_data(docs, chunk_size=500, chunk_overlap=100):
    """
    Chunk large documents into smaller segments.

    Parameters:
    - docs (str or list): Input documents to be chunked. If a single string, it will be treated as one document.
                          If a list of strings, each element will be treated as a separate document.
    - chunk_size (int): Size of each chunk in characters (default is 800).
    - chunk_overlap (int): Number of characters to overlap between adjacent chunks (default is 50).

    Returns:
    - list: List of chunked documents, where each element represents a chunk.
    """

    text_splitter = RecursiveCharacterTextSplitter(chunk_size=chunk_size, chunk_overlap=chunk_overlap)
    chunked_docs = text_splitter.split_documents(docs)
    return chunked_docs

In [6]:
# Loading the PDF
loader = PyPDFLoader("../Sample_docs/Yuvraj_Singh.pdf")
docs = loader.load_and_split()

# Calling the function to create chunks
chunks = chunk_data(docs=docs)

In [7]:
chunks

[Document(page_content='Yuvraj Singh +91-6239305919\nRoll No.:21BCS6343 ys2002github@gmail.com\nB.TECH(Hons) 21BCS6343@cuchd.in\nArtificial Intelligence and Machine learning Github/yuvraaj2002\nChandigarh University,Gharuan Linkedin/Yuvraj Singh\nEducation\nDegree/Certificate Institute/Board CGPA/Percentage Year\nB.Tech. (CSE) Chandigarh University, Gharuan 8.02 Jul 2021- Jul 2025\nSenior Secondary CBSE Board 91.2% Feb 2021\nExperience\n•Samsung research and development Jan 2024 - Present\nPrism Intern Remote', metadata={'source': '../Sample_docs/Yuvraj_Singh.pdf', 'page': 0}),
 Document(page_content='Experience\n•Samsung research and development Jan 2024 - Present\nPrism Intern Remote\n–Innovated a personal document detection and masking algorithm, enhancing identification accuracy by 35%, en-\nsuring privacy regulation compliance and fortifying data security measures across diverse document types.\n•Wictronix Jun 2023 - Aug 2023\nAIML Developer Intern Remote\n–Implemented YOLO V8 for

In [8]:
index_name = "docquest"
pc = Pinecone(api_key=os.environ['PINECONE_API_KEY'])

if index_name in pc.list_indexes().names():
    print("Vector database is already setup")
else:

    # Creating a new index in the vector database
    pc.create_index(
    name=index_name,
    dimension=768,
    metric="cosine",
    spec=ServerlessSpec(
        cloud='aws',
        region='us-east-1'
    )
)

In [9]:
# Get the index reference
index = pc.Index(index_name)

In [10]:
from langchain_pinecone import PineconeVectorStore

In [11]:
# Storing the vectors into the vector database
pinecone = PineconeVectorStore.from_documents(
        chunks, embedding_model, index_name=index_name, pinecone_api_key=os.environ['PINECONE_API_KEY']
    )

## Retrieving the data from Vector DB

In [16]:
%%time
vdb_contxt_text = pinecone.similarity_search("Is there any project on my resume which utilizes Generative AI technology")[:1]
vdb_contxt_text = vdb_contxt_text[0].page_content
vdb_contxt_text

CPU times: total: 0 ns
Wall time: 2.36 s


'detection accuracy, thus increasing fine collections by 25%.\nProjects\n•InterviewX (AI Tutor)\nDeveloped an AI tutor app using GenAI, computer vision, NLP, and ML to boost job market success rates by 38%. Link.\n–Engineered an AI-driven interview preparation application leveraging generative AI, large language model (LLM),\nand advanced machine learning models. Key functionalities include fake job posting detection (F1 score: 0.86),'

## Getting final response from LLM

In [18]:
def get_answer(vdb_contxt_text, query_text, llm):

    template = f"""
        Given the query '{query_text}', and after reviewing the information retrieved from the vector database:
        {vdb_contxt_text}
        Please provide a concise and informative answer that addresses the query effectively.
    """

    prompt = PromptTemplate(input_variables=[[query_text,vdb_contxt_text]], template=template)
    output_parser = StrOutputParser()
    chain = prompt | llm | output_parser

    response = chain.invoke({"source": f"{vdb_contxt_text}\n\n{query_text}"})
    return response

In [20]:
query_text = "Is there any project on my resume which utilizes Generative AI technology?"
get_answer(vdb_contxt_text,query_text,llm)

'Yes, the "InterviewX (AI Tutor)" project utilizes Generative AI technology, specifically generative AI and large language models (LLMs), to enhance job market success rates.'